## Auxiliary Functions

In [65]:
def udc_mapping(code):
    if code >= 0 and code < 99:
        return "General"
    elif code >=100 and code <199:
        return "Philosophy. Psychology"
    elif code >=200 and code <299:
        return "Religion. Theology"
    elif code >=300 and code <399:
        return "Social Sciences"
    elif code >=400 and code <499:
        return "Vacant"
    elif code >=500 and code <599:
        return "Mathematics. Natural Sciences"
    elif code >=600 and code <699:
        return "Applied Sciences. Medicine, Technology"
    elif code >=700 and code <799:
        return "The Arts. Entertainment. Sport"
    elif code >=800 and code <899:
        return "Linguistics. Literature"
    elif code >=900 and code <999:
        return "Geography. History"

# Import Libraries

In [9]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

# Load Data
* The data is extracted directly from the GitHub repository.
* The different tables are concatenated into a single dataframe.

In [23]:
url='https://github.com/FranciscoFoz/7_Days_of_Code_Alura-Python-Pandas/tree/main/Dia_1-Importando_dados/Datasets/dados_emprestimos?raw=true'
with requests.Session() as req:
        r = req.get(url)
        soup = bs(r.content, 'html.parser')
        target = [f"{url[:18]}{item['href']}" for item in soup.select(
            "a[href$='.csv']")]

seen = set()
all_files = []
for item in target:
    if item not in seen:
        seen.add(item)
        all_files.append(item)
 
df=pd.DataFrame()

for x in all_files:
    path= x +'?raw=true'
    df_aux=pd.read_csv(path)
    df=pd.concat([df,df_aux],ignore_index=True)
             


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2258018 entries, 0 to 2258017
Data columns (total 7 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id_emprestimo         int64  
 1   codigo_barras         object 
 2   data_renovacao        object 
 3   data_emprestimo       object 
 4   data_devolucao        object 
 5   matricula_ou_siape    float64
 6   tipo_vinculo_usuario  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 120.6+ MB


In [25]:
df_books = pd.read_parquet('https://github.com/FranciscoFoz/7_Days_of_Code_Alura-Python-Pandas/raw/main/Dia_1-Importando_dados/Datasets/dados_exemplares.parquet')
df_books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 546237 entries, 0 to 568584
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id_exemplar       546237 non-null  int64 
 1   codigo_barras     546237 non-null  object
 2   colecao           546237 non-null  object
 3   biblioteca        546237 non-null  object
 4   status_material   546237 non-null  object
 5   localizacao       546237 non-null  int64 
 6   registro_sistema  546237 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 33.3+ MB


# Data Preprocessing

Drop unused columns.

In [26]:
df_books=df_books.drop('registro_sistema',axis=1)

Removing duplicates.

In [59]:
print("Number of Duplicates of First DF: ",df.duplicated().sum())

df_clean=df.drop_duplicates();

print("Number of Duplicates of Second DF: ",df_books.duplicated().sum())

df_books_clean=df_books.drop_duplicates();

Number of Duplicates of First DF:  37
Number of Duplicates of Second DF:  0


Data type conversion.

In [60]:
matricula_ou_siape_str = df_clean['matricula_ou_siape'].astype('string').to_frame()
df_clean=df_clean.drop('matricula_ou_siape',axis=1)
df_clean.insert(5, 'matricula_ou_siape', matricula_ou_siape_str)


To improve the readability of the “localizacao” column, the universal decimal classification (UDC) will be used. Therefore, a UDC column will be added to the original dataframe, mapping the values of the “localizacao” column.

The mapping will follow the main classes shown in the table below.

| **Notation** | **Description**                                                                                                                      |
|--------------|--------------------------------------------------------------------------------------------------------------------------------------|
| 000 to 099   | Science and Knowledge. Organization. Computer Science. Information Science. Documentation. Librarianship. Institutions. Publications |
| 100 to 199   | Philosophy. Psychology                                                                                                               |
| 200 to 299   | Religion. Theology                                                                                                                   |
| 300 to 399   | Social Sciences                                                                                                                      |
| 400 to 499   | vacant                                                                                                                               |
| 500 to 599   | Mathematics. Natural Sciences                                                                                                        |
| 600 to 699   | Applied Sciences. Medicine, Technology                                                                                               |
| 700 to 799   | The Arts. Entertainment. Sport                                                                                                       |
| 800 to 899   | Linguistics. Literature                                                                                                              |
| 900 to 999   | Geography. History                                                                                                                   |

First, it's necessary to check that all the values are within the range in the table.

In [64]:
print("Out of Range Values: ",len(df_books_clean.query("localizacao<0 or localizacao>999")))

Out of Range Values:  0


In [72]:
df_books_clean['udc']=list(map(udc_mapping,df_books_clean['localizacao']))

Convert columns to datetime

In [81]:
df_clean['data_emprestimo']=pd.to_datetime(df_clean['data_emprestimo'])
df_clean['data_devolucao']=pd.to_datetime(df_clean['data_devolucao'])
df_clean['data_renovacao']=pd.to_datetime(df_clean['data_renovacao'])